In [48]:
MODEL = "gpt-4o-2024-08-06"
#MODEL = "gpt-4o-mini"
#MODEL = "gpt-3.5-turbo"
#content = ""
prompt = [{"role": "user", "content": content}]
loops = 100
load_file = "2"
save_file = "2"

import os
from openai import OpenAI
import pandas as pd
from IPython.display import display_markdown

client = OpenAI()
df_path = os.path.join(os.getcwd())

In [49]:
prompt

[{'role': 'user',
  'content': "Hello! I'm here to assist you with any questions you might have or information you may need. Feel free to ask about a variety of topics, whether it's related to technology, everyday life, or a specific problem you're facing. How can I help you today?"}]

In [11]:
# Load completions from json, if continuing with a file generated in the past

try:
    with open(load_file, "r") as f:
        completions = pd.read_json(f, lines=True)

# If not, create a new completions database

except FileNotFoundError:
    completions = pd.DataFrame([],columns=['Response', 'Eval', 'Data', 'Instruction'])

In [ ]:
for i in range(loops):
    completion = client.chat.completions.create(
        messages = prompt,
        model = MODEL,
        )
    response = completion.choices[0].message.content

    data = {'Response': response, 'Eval': "", 'Data': {}, 'Instruction': prompt}
    data = pd.DataFrame([data])
    data.at[0, "Data"] = {"Completion":completion.to_dict()}
    completions = pd.concat([completions, data],ignore_index=True)
    completions.to_json(os.path.join(df_path, save_file), lines=True, orient="records")

In [20]:
completions.to_json(os.path.join(df_path, save_file), lines=True, orient="records")

In [18]:
n = 83
completions.loc[n,"Eval"] = "Best, probably. Longest."

In [21]:
s = "str"

In [ ]:
s.startswith

In [19]:
completions.iloc[n]

Response       Hello! I'm here to assist you with any questio...
Eval                                    Best, probably. Longest.
Data           {'Completion': {'id': 'chatcmpl-9wQQOS38fCnzhp...
Instruction    [{'role': 'user', 'content': 'Hello! How can I...
Name: 83, dtype: object

In [46]:
content = completions[completions["Eval"].apply(lambda x: x.startswith("Best"))].iloc[0]["Response"]

In [47]:
content

"Hello! I'm here to assist you with any questions you might have or information you may need. Feel free to ask about a variety of topics, whether it's related to technology, everyday life, or a specific problem you're facing. How can I help you today?"

In [ ]:
completions.sort_values("Response")

In [ ]:
completions.sort_values("Response")["Response"].to_list()